# Qa of Tractography Tutorial

This tutorial provides a basic overview of how to run QA of Tractography manually within Python. 
We begin by checking for dependencies, then we set our input parameters and output file path, then we smiply run.

# Dependencies

In [14]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os

from fury import actor, window, colormap, ui
from dipy.tracking.streamline import Streamlines
from PIL import Image
from matplotlib.colors import LinearSegmentedColormap
from ndmg.stats.qa_tractography import qa_tractography

# Set our input/output path

In [15]:
#Change the stream_path manually, the stream_path is the path for the input streamline.trk file.
stream_path = '/Users/zhenhu/Desktop/qa_fiber/streamlines.trk'
#Change the qa_out_path manually, the qa_out_path is the path for the output QA imgae file.
qa_out_path = '/Users/zhenhu/Desktop/vertical'
#Change the brain_path manually, the brain_path is the path for the reference brain scan, in order to get the scan volume..
brain_path = '/Users/zhenhu/Documents/NDD/Downloads/ndmg_outputs2/sub-0025864/ses-1/dwi/preproc/eddy_corrected_data_reor_RAS_res.nii.gz'

# Run the code

In [16]:
qa_tractography(stream_path,qa_out_path,brain_path)